pytorch関連のインストール

In [1]:
!pip3 install torch torchvision torchtext

    100% |████████████████████████████████| 484.0MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b614000 @  0x7f1f499421c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 13.2MB/s 
    100% |████████████████████████████████| 51kB 17.7MB/s 
    100% |████████████████████████████████| 2.0MB 13.9MB/s 
    100% |████████████████████████████████| 51kB 12.7MB/s 
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext


  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


google driveとの同期

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmpcxz3y1fl/pubring.gpg' created
gpg: /tmp/tmpcxz3y1fl/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 19804 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.6.21-0ubuntu2_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.6.21-0ubuntu2) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.6.2

In [2]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

google drive上に設置した本gitレポジトリのディレクトリに移動

In [0]:
import os
path = "drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch"
os.chdir(path)

In [18]:
!pwd

/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch


ここからモデル学習のメイン処理

In [0]:
import os
import sys
import time
import glob

import torch
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets

# from model_bowman16 import SNLIClassifier
from model import SNLIClassifier
from util import get_args, makedirs

パラメータの設定

In [20]:
sys.argv = [sys.argv[0],
            '--epochs', '10', 
            '--d_embed', '300',
            # '--d_proj', '300', 
            # '--dp_ratio', '0.5', 
            # '--n_layers', '2',
            '--d_hidden', '100', 
            # '--train_embed',
            # '--no-bidirectional', 
            '--no-projection',
            '--word_vectors', 'glove.6B.300d', 
            '--vector_cache', '../input/vector_cache']
args = get_args()
args
# !python train.py --epochs 5 --d_embed 300 --gpu -1 --word_vectors glove.6B.300d

Namespace(batch_size=128, birnn=True, d_embed=300, d_hidden=100, d_proj=300, dev_every=1000, dp_ratio=0.2, epochs=10, fix_emb=True, gpu=0, log_every=50, lower=True, lr=0.001, n_layers=1, projection=False, resume_snapshot='', save_every=1000, save_path='results', vector_cache='../input/vector_cache', word_vectors='glove.6B.300d')

In [0]:
torch.cuda.set_device(args.gpu)

データの読み込み

In [0]:
inputs = data.Field(lower=args.lower)
answers = data.Field(sequential=False, unk_token=None)
train, dev, test = datasets.SNLI.splits(inputs, answers, root="../input")

In [0]:
inputs.build_vocab(train, dev, test)

inputs.vocab.load_vectors(args.word_vectors, cache=args.vector_cache)
# if args.word_vectors:
#     if os.path.isfile(args.vector_cache):
#         inputs.vocab.vectors = torch.load(args.vector_cache)
#     else:
#         inputs.vocab.load_vectors(args.word_vectors)
#         makedirs(os.path.dirname(args.vector_cache))
#         torch.save(inputs.vocab.vectors, args.vector_cache)

answers.build_vocab(train)

In [0]:
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=args.batch_size, device=args.gpu)

In [25]:
answers.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'contradiction': 1, 'entailment': 0, 'neutral': 2})

In [0]:
config = args
config.n_embed = len(inputs.vocab)
config.d_out = len(answers.vocab)
config.n_cells = config.n_layers

# double the number of cells for bidirectional networks
if config.birnn:
    config.n_cells *= 2
    
criterion = nn.CrossEntropyLoss()

In [12]:
if args.resume_snapshot:
    model = torch.load(args.resume_snapshot, map_location=lambda storage, locatoin: storage.cuda(args.gpu))
else:
    model = SNLIClassifier(config)
    if args.word_vectors:
        model.embed.weight.data.copy_(inputs.vocab.vectors)
        model.cuda(args.gpu)

opt = O.Adam(model.parameters(), lr=args.lr, weight_decay=1e-6)
# opt = O.RMSprop(model.parameters(), lr=args.lr, weight_decay=1e-6)

ここから学習処理

In [13]:
iterations = 0
start = time.time()
best_dev_acc = -1
train_iter.repeat = False
header = '  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy'
dev_log_template = ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{:8.6f},{:12.4f},{:12.4f}'.split(','))
log_template =     ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{},{:12.4f},{}'.split(','))
makedirs(args.save_path)
print(header)

for epoch in range(args.epochs):
    train_iter.init_epoch()
    n_correct, n_total = 0, 0
    for batch_idx, batch in enumerate(train_iter):

        # switch model to training mode, clear gradient accumulators
        model.train(); opt.zero_grad()

        iterations += 1

        # forward pass
        answer = model(batch)

        # calculate accuracy of predictions in the current batch
        n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
        n_total += batch.batch_size
        train_acc = 100. * n_correct/n_total

        # calculate loss of the network output with respect to training labels
        loss = criterion(answer, batch.label)

        # backpropagate and update optimizer learning rate
        loss.backward(); opt.step()

        # checkpoint model periodically
        if iterations % args.save_every == 0:
            snapshot_prefix = os.path.join(args.save_path, 'snapshot')
            snapshot_path = snapshot_prefix + '_acc_{:.4f}_loss_{:.6f}_iter_{}_model.pt'.format(train_acc, loss.item(), iterations)
            torch.save(model, snapshot_path)
            for f in glob.glob(snapshot_prefix + '*'):
                if f != snapshot_path:
                    os.remove(f)

        # evaluate performance on validation set periodically
        if iterations % args.dev_every == 0:

            # switch model to evaluation mode
            model.eval(); dev_iter.init_epoch()

            # calculate accuracy on validation set
            n_dev_correct, dev_loss = 0, 0
            with torch.no_grad():
                for dev_batch_idx, dev_batch in enumerate(dev_iter):
                     answer = model(dev_batch)
                     n_dev_correct += (torch.max(answer, 1)[1].view(dev_batch.label.size()) == dev_batch.label).sum().item()
                     dev_loss = criterion(answer, dev_batch.label)
            dev_acc = 100. * n_dev_correct / len(dev)

            print(dev_log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), dev_loss.item(), train_acc, dev_acc))

            # update best valiation set accuracy
            if dev_acc > best_dev_acc:

                # found a model with better validation set accuracy

                best_dev_acc = dev_acc
                snapshot_prefix = os.path.join(args.save_path, 'best_snapshot')
                snapshot_path = snapshot_prefix + '_devacc_{}_devloss_{}__iter_{}_model.pt'.format(dev_acc, dev_loss.item(), iterations)

                # save model, delete previous 'best_snapshot' files
                torch.save(model, snapshot_path)
                for f in glob.glob(snapshot_prefix + '*'):
                    if f != snapshot_path:
                        os.remove(f)

        elif iterations % args.log_every == 0:

            # print progress message
            print(log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), ' '*8, n_correct/n_total*100, ' '*12))




  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy
     5     0        50    50/4292        1% 1.018574               38.2188             
     6     0       100   100/4292        2% 0.979129               41.4375             
     8     0       150   150/4292        3% 1.053169               43.6562             
     9     0       200   200/4292        5% 1.033145               45.8359             
    11     0       250   250/4292        6% 0.845060               47.4562             
    12     0       300   300/4292        7% 0.937273               48.6432             
    14     0       350   350/4292        8% 0.816121               49.5067             
    16     0       400   400/4292        9% 0.860180               50.3965             
    17     0       450   450/4292       10% 0.957315               50.9896             
    19     0       500   500/4292       12% 0.890170               51.6312             
    20     0       550   550/42

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


    47     0      1000  1000/4292       23% 0.847520 1.029116      54.9359      60.5162
    58     0      1050  1050/4292       24% 0.829620               55.2262             
    60     0      1100  1100/4292       26% 0.831445               55.5178             
    62     0      1150  1150/4292       27% 0.875833               55.7629             
    63     0      1200  1200/4292       28% 0.858006               56.0078             
    65     0      1250  1250/4292       29% 0.818906               56.1819             
    66     0      1300  1300/4292       30% 0.772901               56.4171             
    68     0      1350  1350/4292       31% 0.801520               56.6418             
    70     0      1400  1400/4292       33% 0.771547               56.8638             
    71     0      1450  1450/4292       34% 0.811975               57.0894             
    73     0      1500  1500/4292       35% 0.833679               57.2693             
    74     0      1550  1550/429

   180     0      3350  3350/4292       78% 0.855413               61.3377             
   182     0      3400  3400/4292       79% 0.838375               61.4085             
   183     0      3450  3450/4292       80% 0.687270               61.4751             
   185     0      3500  3500/4292       82% 0.704933               61.5359             
   187     0      3550  3550/4292       83% 0.811823               61.5955             
   188     0      3600  3600/4292       84% 0.755627               61.6480             
   190     0      3650  3650/4292       85% 0.848055               61.7068             
   191     0      3700  3700/4292       86% 0.738981               61.7544             
   193     0      3750  3750/4292       87% 0.686799               61.8188             
   194     0      3800  3800/4292       89% 0.854209               61.8674             
   196     0      3850  3850/4292       90% 0.809209               61.9284             
   198     0      3900  3900/429

   292     1      5700  1408/4292       33% 0.703547               67.7585             
   294     1      5750  1458/4292       34% 0.637877               67.7801             
   295     1      5800  1508/4292       35% 0.688760               67.7962             
   297     1      5850  1558/4292       36% 0.643383               67.7912             
   299     1      5900  1608/4292       37% 0.768373               67.8293             
   300     1      5950  1658/4292       39% 0.715172               67.8387             
   315     1      6000  1708/4292       40% 0.752636 0.857364      67.8352      68.6649
   327     1      6050  1758/4292       41% 0.812034               67.8354             
   329     1      6100  1808/4292       42% 0.743096               67.8296             
   330     1      6150  1858/4292       43% 0.861146               67.8422             
   332     1      6200  1908/4292       44% 0.717321               67.8811             
   333     1      6250  1958/429

...

  1562     7     31550  1506/4292       35% 0.602828               74.6146             
  1563     7     31600  1556/4292       36% 0.652895               74.6049             
  1565     7     31650  1606/4292       37% 0.544775               74.5870             
  1566     7     31700  1656/4292       39% 0.634511               74.5693             
  1568     7     31750  1706/4292       40% 0.595426               74.5526             
  1570     7     31800  1756/4292       41% 0.596431               74.5506             
  1571     7     31850  1806/4292       42% 0.545213               74.5843             
  1573     7     31900  1856/4292       43% 0.640265               74.5917             
  1574     7     31950  1906/4292       44% 0.628674               74.5827             
  1589     7     32000  1956/4292       46% 0.504488 0.784016      74.5662      71.5911
  1601     7     32050  2006/4292       47% 0.519363               74.5630             
  1603     7     32100  2056/429

  1672     7     33900  3856/4292       90% 0.622669               74.3288             
  1674     7     33950  3906/4292       91% 0.609400               74.3328             
  1687     7     34000  3956/4292       92% 0.573179 0.750409      74.3361      71.6419
  1700     7     34050  4006/4292       93% 0.698453               74.3284             
  1701     7     34100  4056/4292       95% 0.682439               74.3174             
  1703     7     34150  4106/4292       96% 0.499744               74.3063             
  1704     7     34200  4156/4292       97% 0.742881               74.3077             
  1706     7     34250  4206/4292       98% 0.640647               74.3029             
  1707     7     34300  4256/4292       99% 0.693154               74.3014             
  1711     8     34350    14/4292        0% 0.474473               78.2366             
  1713     8     34400    64/4292        1% 0.559451               76.8311             
  1714     8     34450   114/429

  1797     8     36250  1914/4292       45% 0.547718               75.2457             
  1798     8     36300  1964/4292       46% 0.558954               75.2399             
  1800     8     36350  2014/4292       47% 0.680679               75.2327             
  1801     8     36400  2064/4292       48% 0.555636               75.2294             
  1803     8     36450  2114/4292       49% 0.566923               75.2398             
  1805     8     36500  2164/4292       50% 0.512617               75.2289             
  1806     8     36550  2214/4292       52% 0.647768               75.2075             
  1808     8     36600  2264/4292       53% 0.648613               75.1894             
  1809     8     36650  2314/4292       54% 0.733566               75.1644             
  1811     8     36700  2364/4292       55% 0.625737               75.1593             
  1813     8     36750  2414/4292       56% 0.617173               75.1424             
  1814     8     36800  2464/429

  1895     8     38600  4264/4292       99% 0.585792               74.8619             
  1899     9     38650    22/4292        1% 0.522232               76.1719             
  1900     9     38700    72/4292        2% 0.579118               76.7578             
  1902     9     38750   122/4292        3% 0.507720               76.7610             
  1904     9     38800   172/4292        4% 0.522645               76.7851             
  1905     9     38850   222/4292        5% 0.516797               76.6997             
  1907     9     38900   272/4292        6% 0.557701               76.4735             
  1908     9     38950   322/4292        8% 0.474203               76.4582             
  1923     9     39000   372/4292        9% 0.594043 0.735556      76.4050      71.7131
  1935     9     39050   422/4292       10% 0.571673               76.2589             
  1937     9     39100   472/4292       11% 0.519194               76.2282             
  1938     9     39150   522/429

  2008     9     40950  2322/4292       54% 0.641777               75.6006             
  2021     9     41000  2372/4292       55% 0.615052 0.758484      75.5784      71.1746
  2022     9     41050  2422/4292       56% 0.503163               75.5790             
  2024     9     41100  2472/4292       58% 0.615206               75.5663             
  2026     9     41150  2522/4292       59% 0.745849               75.5424             
  2027     9     41200  2572/4292       60% 0.705874               75.5501             
  2029     9     41250  2622/4292       61% 0.560753               75.5527             
  2030     9     41300  2672/4292       62% 0.625825               75.5365             
  2032     9     41350  2722/4292       63% 0.568757               75.5310             
  2034     9     41400  2772/4292       65% 0.613650               75.5098             
  2035     9     41450  2822/4292       66% 0.453218               75.4942             
  2037     9     41500  2872/429

モデルを読み込んでテストデータで評価

In [0]:
if True:
    path_model_eval = "results_epocs10_d-embed300_d-hidden100_no-projection/best_snapshot_devacc_71.71306644990855_devloss_0.7355560064315796__iter_39000_model.pt"
    model_eval = torch.load(path_model_eval,
                            map_location=lambda storage,
                            locatoin: storage.cuda(args.gpu))

    def evaluate(model, data_iter, data_size):
        log_template = '{:>8.6f}, {:8.6f}, {:12.4f}%'
        model.eval(); data_iter.init_epoch()

        # calculate accuracy on validation set
        n_correct, loss = 0, 0
        with torch.no_grad():
            for batch_idx, batch in enumerate(data_iter):
                answer = model(batch)
                n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
                loss = criterion(answer, batch.label)
        acc = 100. * n_correct / data_size

        print(log_template.format(loss.item(), loss.item(), acc))

    evaluate(model_eval, test_iter, len(test))

google driveとの同期プロセスも完全にkillする

In [14]:
# !kill -9 -1